### Web Scraping de SEO BirdLife para la Recolección de Descripciones de Aves con Nombres Comunes y Científicos

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os

In [ ]:
base_url = 'https://seo.org/guia-de-aves/?_page='
driver = webdriver.Chrome()
for i in range(1, 69): # Hay 68 paginas
    print(f'Starting page {i}...', end='\r')
    driver.get(base_url + str(i))
    birds = driver.find_elements(By.CLASS_NAME, 'titular.noocasional')
    urls = [bird.find_element(By.TAG_NAME, 'a').get_attribute('href') for bird in birds]
    occassional_birds = driver.find_elements(By.CLASS_NAME, 'titular.ocasional')
    urls += [bird.find_element(By.TAG_NAME, 'a').get_attribute('href') for bird in occassional_birds]
    common_names = []
    scientific_names = []
    descriptions = []
    for url in urls:
        driver.get(url)
        common_name = driver.find_element(By.CLASS_NAME, 'fusion-title-heading.title-heading-left.fusion-responsive-typography-calculated').text
        scientific_name = driver.find_element(By.CLASS_NAME, 'fusion-text.fusion-text-1.fusion-text-no-margin.HindMadurai-Regular.italichind').text
        try:
            description = driver.find_element(By.CLASS_NAME, 'fusion-text.fusion-text-12.HindMadurai-Regular').text
            if len(description) == 0:
                raise Exception
        except:
            try:
                description = driver.find_element(By.CLASS_NAME, 'fusion-text.fusion-text-20.HindMadurai-Regular').text
            except:
                print(f'Error finding description of {common_name}')
                continue
        description = description.replace('Identificación', '')
        description = description.replace('\n', ' ').strip()
        common_names.append(common_name)
        scientific_names.append(scientific_name)
        descriptions.append(description)
    df = pd.DataFrame({'Nombres comunes': common_names, 'Nombre aceptado': scientific_names, 'Descripción': descriptions})
    df['Grupo taxonómico'] = 'Aves'
    df['Fuente'] = 'SEO BirdLife'
    os.makedirs('Datos SEO BirdLife', exist_ok=True)
    df.to_excel('Datos SEO BirdLife/birds_descriptions_SEO.xlsx', index=False)
driver.close()